In [1]:
import xlrd
import pandas as pd
import numpy as np
import json
import unidecode
from functools import reduce

In [2]:
df = pd.read_excel('input/youth_survey_raw.xlsx')

# get question ids from columns
question_ids = [ c for c in df.columns if 'Q' in c ]

# copy row 0 to column names
df.columns = df.iloc[0]

# delete row 0
df = df.drop(0, axis=0)
df = df.applymap(str)




In [3]:
question_columns = [col.strip() for col in df.columns if col.startswith(('Q'))]
demographic_columns = [col.strip() for col in df.columns if col.startswith(('S'))]
demographic_columns

['S1: How old are you',
 'S2: What is your marital status?',
 'S3: Do you have any children?',
 'S4: (If "yes"), How many?',
 'S5: Including you, how many people, children, above and under 15 live in this household?',
 'S6: What is the highest education level you completed?',
 'S7: Which one of the folowing income is close to your household monthly income, including all household members incomes live in or out this house?',
 'S8: What is your job status today?',
 'S11: Gender of respondent']

In [4]:
df_ = df.filter(items=question_columns)
df_['governorate'] = df['M2: Governorate']
df_['district'] = df['M3: District']
df_['urbanity'] = df['M7: Urbanity']
df_['household_monthly_income'] = df['S7: Which one of the folowing income is close to your household monthly income, including all household members incomes live in or out this house?']
df_['age'] = df['S1: How old are you']
df_['marital_status'] = df['S2: What is your marital status?'].apply(lambda x: x.split(', ')[0])
df_['have_children'] = df['S3: Do you have any children?']
df_['job_status'] = df['S8: What is your job status today?'].apply(lambda x: x.replace('Unemployed, looking for job', 'Unemployed').replace('Works in his/her own business', 'Self-employed'))
df_['gender'] = df['S11: Gender of respondent']

df_.head()

,Q1: First I would like to ask you about your current situation / family situation. How would you rate it on a scale from 1 to 5?,"Q2: In general, what do you think about the current situation in Yemen, is it going in the right direction or in the wrong direction?",Q3: What is your status:,Q3x: Other,"Q4_1: What are the reasons you had to stop going to school, university, vocational training or lost your job? First answer",Q4_1_X :Other,"Q4_2: What are the reasons you had to stop going to school, university, vocational training or lost your job? Second answer",Q4_2_X :Other,Q5: How much time do you spend at school/university/work/vocational training?,Q6: How close is the school/university/work/vocational training you attend to your house?,...,Q61_3_X: Other,governorate,district,urbanity,household_monthly_income,age,marital_status,have_children,job_status,gender
1,Neither bad nor good,In a very bad direction,School student,nan,nan,nan,nan,nan,5-8 hours,0-1 km,...,nan,Ibb,Al-Sabrah,Rural,"20,000 YER- 39,000 YER",17,Single,nan,Student,Male
2,It’s very bad,"3.Not in the right direction, not in the wrong...",School student,nan,nan,nan,nan,nan,1-4 hours,0-1 km,...,nan,Ibb,Al-Makhader,Rural,"20,000 YER- 39,000 YER",16,Single,nan,Student,Male
3,It’s very bad,In a very bad direction,I am a housewife,nan,nan,nan,nan,nan,nan,nan,...,nan,Abyan,Khanfar,Rural,"Less than 20,000 YER",25,Married,Yes,Housewife,Female
4,Neither bad nor good,"3.Not in the right direction, not in the wrong...",I am a housewife,nan,nan,nan,nan,nan,nan,nan,...,nan,Shabwah,Jardaan,Rural,"60,000 YER - 79,000 YER",19,Married,No,Housewife,Female
5,It’s somewhat good,In a very bad direction,School student,nan,nan,nan,nan,nan,1-4 hours,0-1 km,...,nan,ad-Dali,Qatabah,Rural,"60,000 YER - 79,000 YER",18,Single,nan,Student,Male


In [24]:
df_a_list = []

idMap = {
    "abyan":"2",
    "ad-dali":"19",
    "aden":"13",
    "al-baidha":"4",
    "al-mahra":"17",
    "al-mahweet":"16",
    "alhudaida":"8",
    "aljawf":"6",
    "amran":"18",
    "dhamar":"10",
    "hadhramawt":"9",
    "hajja":"7",
    "ibb":"1",
    "lahj":"14",
    "marib":"15",
    "rayma":"20",
    "sana'a city":"3",
    "sana'a":"12",
    "shabwah":"11",
    "taiz":"5"
}

for q in question_columns:
    df_a = pd.DataFrame()
    
    
    
    q_ = unidecode.unidecode(q)
    qID = q_.split(':')[0]
    q_ = q_.split(':')[1].strip()
    
    
  
    if q_ != 'Other':
        df_a['id'] = df_['governorate'].apply(lambda x: idMap[str(x).lower()])
        df_a['governorate'] = df_['governorate']
        df_a['district'] = df_['district']
        df_a['qID'] = qID
        df_a['question_raw'] = q_
        df_a['answer_raw'] = df_[q]
        df_a['urbanity'] = df_['urbanity']
        df_a['household_monthly_income'] = df_['household_monthly_income']
        df_a['age'] = df_['age']
        df_a['marital_status'] = df_['marital_status']
        df_a['have_children'] = df_['have_children']
        df_a['job_status'] = df_['job_status']
        df_a['gender'] = df_['gender']
    
    
    

    df_a_list.append(df_a)

df_formatted = pd.concat(df_a_list)


df_formatted.head()

    

,id,governorate,district,qID,question_raw,answer_raw,urbanity,household_monthly_income,age,marital_status,have_children,job_status,gender
1,1,Ibb,Al-Sabrah,Q1,First I would like to ask you about your curre...,Neither bad nor good,Rural,"20,000 YER- 39,000 YER",17,Single,nan,Student,Male
2,1,Ibb,Al-Makhader,Q1,First I would like to ask you about your curre...,It’s very bad,Rural,"20,000 YER- 39,000 YER",16,Single,nan,Student,Male
3,2,Abyan,Khanfar,Q1,First I would like to ask you about your curre...,It’s very bad,Rural,"Less than 20,000 YER",25,Married,Yes,Housewife,Female
4,11,Shabwah,Jardaan,Q1,First I would like to ask you about your curre...,Neither bad nor good,Rural,"60,000 YER - 79,000 YER",19,Married,No,Housewife,Female
5,19,ad-Dali,Qatabah,Q1,First I would like to ask you about your curre...,It’s somewhat good,Rural,"60,000 YER - 79,000 YER",18,Single,nan,Student,Male


In [25]:
answer_count = df_formatted.groupby(['governorate', 'question_raw', 'answer_raw']).transform('count')
answer_total = df_formatted.groupby(['governorate', 'question_raw']).transform('count')

df_formatted['answer_count'] = ""
df_formatted['answer_count'] = answer_count

df_formatted['answer_total'] = ""
df_formatted['answer_total'] = answer_total

df_formatted['answer_pct'] = (df_formatted['answer_count'] / df_formatted['answer_total'])*100

df_formatted.head()

,id,governorate,district,qID,question_raw,answer_raw,urbanity,household_monthly_income,age,marital_status,have_children,job_status,gender,answer_count,answer_total,answer_pct
1,1,Ibb,Al-Sabrah,Q1,First I would like to ask you about your curre...,Neither bad nor good,Rural,"20,000 YER- 39,000 YER",17,Single,nan,Student,Male,61,170,35.882353
2,1,Ibb,Al-Makhader,Q1,First I would like to ask you about your curre...,It’s very bad,Rural,"20,000 YER- 39,000 YER",16,Single,nan,Student,Male,53,170,31.176471
3,2,Abyan,Khanfar,Q1,First I would like to ask you about your curre...,It’s very bad,Rural,"Less than 20,000 YER",25,Married,Yes,Housewife,Female,13,30,43.333333
4,11,Shabwah,Jardaan,Q1,First I would like to ask you about your curre...,Neither bad nor good,Rural,"60,000 YER - 79,000 YER",19,Married,No,Housewife,Female,16,40,40.000000
5,19,ad-Dali,Qatabah,Q1,First I would like to ask you about your curre...,It’s somewhat good,Rural,"60,000 YER - 79,000 YER",18,Single,nan,Student,Male,9,40,22.500000


In [26]:
xls = pd.ExcelFile('tagged.xlsx')
tagged_questions = pd.read_excel(xls, 'questions')
tagged_answers = pd.read_excel(xls, 'answers')


df_tagged = tagged_questions.merge(tagged_answers, on = ['qID', 'question_raw'], how = 'outer')
df_tagged

# df_final = reduce(lambda left,right: pd.merge(left,right,on=['qID', 'question_raw']), [df_formatted, tagged_questions, tagged_answers])


,qID,question_raw,question_en,question_ar,question_tag,question_tag_order,question_map,question_keep,question_type,question_scale,aID,answer_raw,answer_en,answer_ar,answer_rank
0,Q1,First I would like to ask you about your curre...,How would you rate you and your family's curre...,NaN,daily life,1.0,y,y,scale,5.0,Q1_a1,Don't know,NaN,NaN,dk
1,Q1,First I would like to ask you about your curre...,How would you rate you and your family's curre...,NaN,daily life,1.0,y,y,scale,5.0,Q1_a2,It is very good,Very good,NaN,5
2,Q1,First I would like to ask you about your curre...,How would you rate you and your family's curre...,NaN,daily life,1.0,y,y,scale,5.0,Q1_a3,It's somewhat bad,Bad,NaN,2
3,Q1,First I would like to ask you about your curre...,How would you rate you and your family's curre...,NaN,daily life,1.0,y,y,scale,5.0,Q1_a4,It's somewhat good,Good,NaN,4
4,Q1,First I would like to ask you about your curre...,How would you rate you and your family's curre...,NaN,daily life,1.0,y,y,scale,5.0,Q1_a5,It's very bad,Very bad,NaN,1
5,Q1,First I would like to ask you about your curre...,How would you rate you and your family's curre...,NaN,daily life,1.0,y,y,scale,5.0,Q1_a6,Neither bad nor good,Average,NaN,3
6,Q2,"In general, what do you think about the curren...",Is the situation in Yemen heading in the right...,NaN,politics,1.0,y,y,scale,5.0,Q2_a1,Don't know,NaN,NaN,dk
7,Q2,"In general, what do you think about the curren...",Is the situation in Yemen heading in the right...,NaN,politics,1.0,y,y,scale,5.0,Q2_a2,"Not in the right direction, not in the wrong d...",Neither the right nor the wrong direction,NaN,3
8,Q2,"In general, what do you think about the curren...",Is the situation in Yemen heading in the right...,NaN,politics,1.0,y,y,scale,5.0,Q2_a3,In a very bad direction,The wrong direction,NaN,1
9,Q2,"In general, what do you think about the curren...",Is the situation in Yemen heading in the right...,NaN,politics,1.0,y,y,scale,5.0,Q2_a4,In the right direction,The right direction,NaN,5


In [27]:
df_final = df_formatted.merge(df_tagged, on = ['qID', 'question_raw', 'answer_raw'], how = 'outer')
df_final = df_final[df_final['question_keep'] == 'y']
print(list(df_final.columns), len(list(df_final.columns)))

df_final = df_final[['id','governorate', 'district', 'qID', 'question_raw', 'question_en', 'question_ar', 'question_tag', 'question_tag_order', 'question_type', 'question_map', 'question_scale', 'answer_raw', 'answer_en', 'answer_ar', 'answer_count', 'answer_total', 'answer_pct', 'answer_rank', 'urbanity', 'household_monthly_income', 'age', 'marital_status', 'job_status', 'gender', 'have_children']]
df_final.head()

['id', 'governorate', 'district', 'qID', 'question_raw', 'answer_raw', 'urbanity', 'household_monthly_income', 'age', 'marital_status', 'have_children', 'job_status', 'gender', 'answer_count', 'answer_total', 'answer_pct', 'question_en', 'question_ar', 'question_tag', 'question_tag_order', 'question_map', 'question_keep', 'question_type', 'question_scale', 'aID', 'answer_en', 'answer_ar', 'answer_rank'] 28


,id,governorate,district,qID,question_raw,question_en,question_ar,question_tag,question_tag_order,question_type,...,answer_total,answer_pct,answer_rank,urbanity,household_monthly_income,age,marital_status,job_status,gender,have_children
0,1,Ibb,Al-Sabrah,Q1,First I would like to ask you about your curre...,How would you rate you and your family's curre...,NaN,daily life,1.0,scale,...,170.0,35.882353,3,Rural,"20,000 YER- 39,000 YER",17,Single,Student,Male,nan
1,11,Shabwah,Jardaan,Q1,First I would like to ask you about your curre...,How would you rate you and your family's curre...,NaN,daily life,1.0,scale,...,40.0,40.000000,3,Rural,"60,000 YER - 79,000 YER",19,Married,Housewife,Female,No
2,9,Hadhramawt,Al-Mukala City,Q1,First I would like to ask you about your curre...,How would you rate you and your family's curre...,NaN,daily life,1.0,scale,...,80.0,21.250000,3,Urban,(DK),16,Single,Student,Female,nan
3,1,Ibb,Al-Sabrah,Q1,First I would like to ask you about your curre...,How would you rate you and your family's curre...,NaN,daily life,1.0,scale,...,170.0,35.882353,3,Rural,"40,000 YER - 59,000 YER",25,Widow,Housewife,Female,Yes
4,1,Ibb,Thee Al-Sufal,Q1,First I would like to ask you about your curre...,How would you rate you and your family's curre...,NaN,daily life,1.0,scale,...,170.0,35.882353,3,Rural,"60,000 YER - 79,000 YER",17,Single,Student,Female,nan


In [34]:
df_final.to_csv('final.csv', index=False)

    

In [92]:
df_final.head()


# df.drop_duplicates(subset=['A','B'])

df_final_filtered = df_final[['id', 'governorate', 'question_tag', 'question_tag_order', 'question_raw', 'question_en', 'question_ar', 'answer_raw', 'answer_en', 'answer_ar', 'answer_rank', 'answer_count', 'answer_pct']]
df_final_filtered = df_final_filtered.sort_values(['id', 'question_tag', 'question_tag_order', 'answer_rank'])
df_final_filtered = df_final_filtered.drop_duplicates()
df_final_filtered = df_final_filtered[df_final_filtered['answer_rank'] != 'removed / repeated']


# df_final_filtered.sort_values(['id','question_tag', 'question_tag_order', 'answer_rank']).to_csv('filtered.csv', index=False)
df_final_filtered.to_csv('filtered.csv', index=False)

In [199]:
for key, df_ in df_final_filtered.groupby( ['id'] ):
    
    print (df_)
    
#     df_.to_csv('output/'+key+'.csv', index=False)
    
    
#     print ('____')
#     print (key)
    
#     overall = []
    
    
#     for index, df__ in df_.groupby(['question_raw']):
#         print (df_['question_en'])
        
#         dict_ = {
#             'id': key,
#             'question_raw': index,
#         }
#         overall.append(dict_)
        
#         print ()
#         print (dict_)

    

        
       
    
    
        
    
    
            
    

       id governorate      question_tag  question_tag_order  \
117714  1         Ibb       basic needs                 1.0   
118434  1         Ibb       basic needs                 1.0   
117786  1         Ibb       basic needs                 1.0   
116976  1         Ibb       basic needs                 1.0   
115804  1         Ibb       basic needs                 1.0   
124799  1         Ibb       basic needs                 2.0   
126751  1         Ibb       basic needs                 2.0   
219357  1         Ibb       basic needs                 3.0   
219463  1         Ibb       basic needs                 3.0   
218873  1         Ibb       basic needs                 3.0   
218581  1         Ibb       basic needs                 3.0   
221172  1         Ibb       basic needs                 4.0   
220081  1         Ibb       basic needs                 4.0   
220788  1         Ibb       basic needs                 4.0   
221452  1         Ibb       basic needs                

        id governorate      question_tag  question_tag_order  \
118752  16  al-Mahweet       basic needs                 1.0   
118366  16  al-Mahweet       basic needs                 1.0   
117574  16  al-Mahweet       basic needs                 1.0   
116858  16  al-Mahweet       basic needs                 1.0   
126483  16  al-Mahweet       basic needs                 2.0   
127667  16  al-Mahweet       basic needs                 2.0   
219451  16  al-Mahweet       basic needs                 3.0   
219980  16  al-Mahweet       basic needs                 3.0   
219294  16  al-Mahweet       basic needs                 3.0   
218862  16  al-Mahweet       basic needs                 3.0   
221435  16  al-Mahweet       basic needs                 4.0   
220671  16  al-Mahweet       basic needs                 4.0   
221134  16  al-Mahweet       basic needs                 4.0   
224048  16  al-Mahweet       basic needs                 5.0   
224527  16  al-Mahweet       basic needs

       id governorate      question_tag  question_tag_order  \
117736  5        Taiz       basic needs                 1.0   
118522  5        Taiz       basic needs                 1.0   
117976  5        Taiz       basic needs                 1.0   
117110  5        Taiz       basic needs                 1.0   
116142  5        Taiz       basic needs                 1.0   
125293  5        Taiz       basic needs                 2.0   
127027  5        Taiz       basic needs                 2.0   
219414  5        Taiz       basic needs                 3.0   
219632  5        Taiz       basic needs                 3.0   
218974  5        Taiz       basic needs                 3.0   
218645  5        Taiz       basic needs                 3.0   
221232  5        Taiz       basic needs                 4.0   
220269  5        Taiz       basic needs                 4.0   
220909  5        Taiz       basic needs                 4.0   
221466  5        Taiz       basic needs                

In [235]:
question_cols = ['id', 'governorate', 'question_tag', 'question_tag_order', 'question_raw', 'question_en', 'question_ar']
answer_cols = ['answer_raw', 'answer_en', 'answer_ar', 'answer_rank', 'answer_count', 'answer_pct']

def build_question_dict(df_):
    result = {
        c: df_[c].values[0] for c in question_cols
    }

    result['values'] = []
    for _, row in df_.groupby('answer_raw'):

        result['values'].append({
            c: row[c].values[0] for c in answer_cols        
        })
        
    return result


# df_ = df_final_filtered.loc[ (df_final_filtered.id == '1') & (df_final_filtered.question_raw == 'Please tell me if Adequate  shelter/housing is available or not for your family')]
# df_ = df_final_filtered.loc[ (df_final_filtered.id == '1') ]

results = []
for id_, df_ in df_final_filtered.groupby('question_raw'):
    results.append( build_question_dict(df_) )

print(json.dumps(results, indent=2))
        

[
  {
    "id": "1",
    "governorate": "Ibb",
    "question_tag": "public services",
    "question_tag_order": 5.0,
    "question_raw": "(If yes) How many times?",
    "question_en": "If yes, how many times has your family received assistance?",
    "question_ar": NaN,
    "values": [
      {
        "answer_raw": "1",
        "answer_en": 1,
        "answer_ar": NaN,
        "answer_rank": 1,
        "answer_count": 19.0,
        "answer_pct": 11.176470588235295
      },
      {
        "answer_raw": "10",
        "answer_en": "Other",
        "answer_ar": NaN,
        "answer_rank": "x",
        "answer_count": 2.0,
        "answer_pct": 1.6666666666666667
      },
      {
        "answer_raw": "12",
        "answer_en": NaN,
        "answer_ar": NaN,
        "answer_rank": "x",
        "answer_count": 1.0,
        "answer_pct": 2.5
      },
      {
        "answer_raw": "14",
        "answer_en": NaN,
        "answer_ar": NaN,
        "answer_rank": "x",
        "answer_count": 1.0

In [220]:
df_

,id,governorate,question_tag,question_tag_order,question_raw,question_en,question_ar,answer_raw,answer_en,answer_ar,answer_rank,answer_count,answer_pct
117714,1,Ibb,basic needs,1.0,Please tell me if Adequate shelter/housing is...,Does your family have access to adequate shelt...,NaN,Not available at all,"No, never",NaN,1,1.0,0.588235
118434,1,Ibb,basic needs,1.0,Please tell me if Adequate shelter/housing is...,Does your family have access to adequate shelt...,NaN,Somewhat not available,Only sometimes,NaN,2,7.0,4.117647
117786,1,Ibb,basic needs,1.0,Please tell me if Adequate shelter/housing is...,Does your family have access to adequate shelt...,NaN,Somewhat available,Sometimes yes,NaN,3,42.0,24.705882
116976,1,Ibb,basic needs,1.0,Please tell me if Adequate shelter/housing is...,Does your family have access to adequate shelt...,NaN,Available,Yes,NaN,4,39.0,22.941176
115804,1,Ibb,basic needs,1.0,Please tell me if Adequate shelter/housing is...,Does your family have access to adequate shelt...,NaN,Very available,"Yes, always",NaN,5,81.0,47.647059
